<a href="https://colab.research.google.com/github/devesssi/dl-playground/blob/main/FER2013_CNN_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os


In [2]:
!pip install opendatasets

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/msambare/fer2013")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: tiwaridevesh
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013


100%|██████████| 60.3M/60.3M [00:00<00:00, 429MB/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# transforms: images ko resize + tensor mein convert
transform = transforms.Compose([
    transforms.Grayscale(),  # agar dataset gray hai, warna hata de
    transforms.Resize((48, 48)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root="fer2013/train", transform=transform)
test_data  = datasets.ImageFolder(root="fer2013/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=64)


In [5]:
import torch.nn as nn

class FER_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),   # (1, 48, 48) → (32, 48, 48)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),              # → (32, 24, 24)

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),             # → (64, 12, 12)
        )
        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*12*12, 128),
            nn.ReLU(),
            nn.Linear(128, 7)  # FER-2013 has 7 classes
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = self.fc_layer(x)
        return x

model = FER_CNN()


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
for epoch in range(10):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
# executed in 1157.668s

In [8]:
def evaluate(model, dataloader, loss_fn, device):
    model.eval()  # eval mode
    test_loss, correct = 0, 0

    with torch.no_grad():  # no gradients here
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            preds = model(X)
            test_loss += loss_fn(preds, y).item()
            correct += (preds.argmax(1) == y).sum().item()

    avg_loss = test_loss / len(dataloader)
    accuracy = correct / len(dataloader.dataset)

    return avg_loss, accuracy


In [9]:
import matplotlib.pyplot as plt

def predict(model, image, classes, device):
    model.eval()
    with torch.no_grad():
        image = image.unsqueeze(0).to(device)  # (C, H, W) → (1, C, H, W)
        preds = model(image)
        pred_class = preds.argmax(1).item()

    plt.imshow(image.squeeze().cpu(), cmap='gray')
    plt.title(f"Predicted: {classes[pred_class]}")
    plt.show()


In [10]:
torch.save(model.state_dict(), "fer_cnn.pth")
print("✅ Model saved!")


✅ Model saved!


In [13]:
import torch

# 1. Model define karo (same architecture)
model = FER_CNN()
# Load the state dictionary from the correct file
model.load_state_dict(torch.load("fer_cnn.pth"))

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# 2. Evaluate the loaded model
# Assuming you have 'test_loader', 'criterion', and 'device' defined
# If not, you'll need to define them before this step.
test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Test Loss: 1.3755, Test Accuracy: 0.5155
